In [0]:
df = spark.read.table("main_prod.datascience_scratchpad.nyc_train_filtered")
display(df)

In [0]:
query = """
select * from main_prod.datascience_scratchpad.nyc_train_filtered where traj_id rlike '16808692_*'
"""

df = spark.sql(query)
display(df)

In [0]:
from pyspark.sql.functions import col
lon_lat_list = df.filter(col('traj_id')=='16808692_2024-01-04').select('wgs_seq_filtered').collect()[0][0]
print(lon_lat_list)

In [0]:
# userid = '4474029'
# start_time = '2025-07-02'
# origin_types = [0,1,2,3]
# pip_list = []
# for origin_type in origin_types:
#     query = f"""select * from main_prod.datascience.userpiphistory where userid = {userid} and createdon <= '{start_time}' and placetype = 'Job' and origintype = {origin_type} order by createdon desc limit 1"""
#     df_userpip = spark.sql(query).select('lat','lng')
#     if df_userpip.count()>0:
#         lat = df_userpip.collect()[0]['lat']
#         long = df_userpip.collect()[0]['lng']
#         pip_list.append((lat,long))

In [0]:
# !pip install folium
import folium

from folium import plugins
coordinates = [[x[1],x[0]] for x in lon_lat_list]
# Initialize map
m = folium.Map(location=coordinates[0], zoom_start=6)


# for lat,lon in pip_list:
#     folium.Marker(
#         location=[lat, lon],
#         popup="Pip Location",
#         icon=folium.Icon(color='red')  # Color can be 'red', 'blue', 'green', 'purple', etc.
#     ).add_to(m)

folium.Marker(
    location=coordinates[0],
    popup="Start Location",
    icon=folium.Icon(color='green')  # Color can be 'red', 'blue', 'green', 'purple', etc.
).add_to(m)
# Add markers
for i, (lat, lon) in enumerate(coordinates[1:]):
    folium.Marker([lat, lon], popup=i+1).add_to(m)

# Draw arrows between points
for i in range(len(coordinates) - 1):
    start = coordinates[i]
    end = coordinates[i + 1]

    # Draw the line
    line = folium.PolyLine([start, end], color="blue", weight=3, opacity=0.7).add_to(m)

    # Add directional arrow
    plugins.PolyLineTextPath(
        line,
        '➤',  # arrow symbol
        repeat=True,
        offset=7,
        attributes={'fill': 'blue', 'font-weight': 'bold', 'font-size': '16'}
    ).add_to(m)

# Save and show the map
m.save("map_with_arrows.html")
m


In [0]:
import pickle

cs= pickle.load(open("/Volumes/main_prod/datascience_scratchpad/jatin/trajcl_exp/nyc/nyc_cell500_cellspace.pkl", "rb"))

In [0]:
import math
def lonlat2meters(lon, lat):
    semimajoraxis = 6378137.0
    east = lon * 0.017453292519943295
    north = lat * 0.017453292519943295
    t = math.sin(north)
    return semimajoraxis * east, 3189068.5 * math.log((1 + t) / (1 - t))

In [0]:

point_ids = [(i,cs.get_cellid_by_point(*lonlat2meters(x[0],x[1])), x) for i,x in enumerate(lon_lat_list)]
point_ids

## Porto Data Vis

In [0]:
lon_lat_list =[(40.43911064901829, 40.376522035126264),
 (40.44780198235162, 40.288086045348),
 (40.3824409823516, 40.230664454492626),
 (40.342563856443874, 40.248130302468695),
 (40.35758431568494, 40.27359789534808),
 (40.35630348235161, 40.315881900486346),
 (40.380179284257196, 40.32517491711977),
 (40.40357431568497, 40.34084069884953),
 (40.41461936583309, 40.35091932028082),
 (40.41489555879487, 40.36919507634151),
 (40.42838614901827, 40.37722384546589),
 (40.43970726441913, 40.38281022063537),
 (40.436605212794866, 40.379722282277925),
 (40.43667937203741, 40.375511426506286),
 (40.43239155993069, 40.38146275897038),
 (40.40361991254236, 40.38429803867161),
 (40.40902188055657, 40.383905030564044)]

In [0]:
!pip install folium
import folium

from folium import plugins
coordinates = [[x[1],x[0]] for x in lon_lat_list]
# Initialize map
m = folium.Map(location=coordinates[0], zoom_start=6)


# for lat,lon in pip_list:
#     folium.Marker(
#         location=[lat, lon],
#         popup="Pip Location",
#         icon=folium.Icon(color='red')  # Color can be 'red', 'blue', 'green', 'purple', etc.
#     ).add_to(m)

folium.Marker(
    location=coordinates[0],
    popup="Start Location",
    icon=folium.Icon(color='green')  # Color can be 'red', 'blue', 'green', 'purple', etc.
).add_to(m)
# Add markers
for lat, lon in coordinates[1:]:
    folium.Marker([lat, lon]).add_to(m)

# Draw arrows between points
for i in range(len(coordinates) - 1):
    start = coordinates[i]
    end = coordinates[i + 1]

    # Draw the line
    line = folium.PolyLine([start, end], color="blue", weight=3, opacity=0.7).add_to(m)

    # Add directional arrow
    plugins.PolyLineTextPath(
        line,
        '➤',  # arrow symbol
        repeat=True,
        offset=7,
        attributes={'fill': 'blue', 'font-weight': 'bold', 'font-size': '16'}
    ).add_to(m)

# Save and show the map
m.save("map_with_arrows.html")
m
